In [1]:
#necessary libraries
import pandas as pd
import sqlite3

In [3]:
#load the dataset
df = pd.read_csv("/content/Superstore.csv", encoding='ISO-8859-1')

#create the SQLite in-memory database
conn = sqlite3.connect(":memory:")

# Load the DataFrame into SQLite
df.to_sql("superstore", conn, index=False, if_exists="replace")

9994

In [4]:
# Query 1: Customer order count and total revenue
query1 = """
SELECT
    [Customer Name] AS customer,
    COUNT(DISTINCT [Order ID]) AS total_orders,
    SUM(Sales) AS total_revenue
FROM superstore
GROUP BY [Customer Name]
ORDER BY total_revenue DESC;
"""

In [5]:
# Query 2: Top 5 highest spending customers with their country
query2 = """
SELECT
    [Customer Name] AS customer,
    MAX(Country) AS country,
    SUM(Sales) AS total_revenue
FROM superstore
GROUP BY [Customer Name]
ORDER BY total_revenue DESC
LIMIT 5;
"""

In [6]:
# Query 3: Product categories and total revenue
query3 = """
SELECT
    Category,
    SUM(Sales) AS total_revenue
FROM superstore
GROUP BY Category
ORDER BY total_revenue DESC;
"""

# Execute and return results
result1 = pd.read_sql_query(query1, conn)
result2 = pd.read_sql_query(query2, conn)
result3 = pd.read_sql_query(query3, conn)

result1.head(), result2, result3

(        customer  total_orders  total_revenue
 0    Sean Miller             5      25043.050
 1   Tamara Chand             5      19052.218
 2   Raymond Buch             6      15117.339
 3   Tom Ashbrook             4      14595.620
 4  Adrian Barton            10      14473.571,
         customer        country  total_revenue
 0    Sean Miller  United States      25043.050
 1   Tamara Chand  United States      19052.218
 2   Raymond Buch  United States      15117.339
 3   Tom Ashbrook  United States      14595.620
 4  Adrian Barton  United States      14473.571,
           Category  total_revenue
 0       Technology    836154.0330
 1        Furniture    741999.7953
 2  Office Supplies    719047.0320)

In [7]:
# Query 1: Rank each customer's orders by order amount using ROW_NUMBER()
query1 = """
SELECT
    [Customer Name],
    [Order ID],
    Sales,
    ROW_NUMBER() OVER (PARTITION BY [Customer Name] ORDER BY Sales DESC) AS order_rank
FROM superstore;
"""

# Query 2: Running total of sales by month
query2 = """
SELECT
    strftime('%Y-%m', [Order Date]) AS order_month,
    Sales,
    SUM(Sales) OVER (PARTITION BY strftime('%Y-%m', [Order Date]) ORDER BY [Order Date]) AS monthly_running_total
FROM superstore;
"""

# Query 3: Top 3 products per category by revenue using RANK()
query3 = """
SELECT *
FROM (
    SELECT
        Category,
        [Product Name],
        SUM(Sales) AS total_sales,
        RANK() OVER (PARTITION BY Category ORDER BY SUM(Sales) DESC) AS sales_rank
    FROM superstore
    GROUP BY Category, [Product Name]
)
WHERE sales_rank <= 3;
"""

# Execute queries
result1 = pd.read_sql_query(query1, conn)
result2 = pd.read_sql_query(query2, conn)
result3 = pd.read_sql_query(query3, conn)

result1.head(), result2.head(), result3


(   Customer Name        Order ID    Sales  order_rank
 0  Aaron Bergman  CA-2016-140935  341.960           1
 1  Aaron Bergman  CA-2014-156587  242.940           2
 2  Aaron Bergman  CA-2016-140935  221.980           3
 3  Aaron Bergman  CA-2014-156587   48.712           4
 4  Aaron Bergman  CA-2014-156587   17.940           5,
   order_month    Sales  monthly_running_total
 0        None   48.896               1481.828
 1        None  474.430               1481.828
 2        None    3.600               1481.828
 3        None  454.560               1481.828
 4        None  141.420               1481.828,
           Category                                       Product Name  \
 0        Furniture       HON 5400 Series Task Chairs for Big and Tall   
 1        Furniture  Riverside Palais Royal Lawyers Bookcase, Royal...   
 2        Furniture         Bretford Rectangular Conference Table Tops   
 3  Office Supplies  Fellowes PB500 Electric Punch Plastic Comb Bin...   
 4  Office Suppl

In [8]:
# Convert 'Order Date' to datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

# Drop rows with invalid dates
df = df.dropna(subset=['Order Date'])

# CTE 1: Customers with average order value > ₹1000
cte_avg_order_value = (
    df.groupby('Customer Name')['Sales']
    .mean()
    .reset_index()
    .rename(columns={'Sales': 'avg_order_value'})
)
cte_avg_order_value = cte_avg_order_value[cte_avg_order_value['avg_order_value'] > 1000]

# CTE 2: Orders in the last 3 months, then aggregate revenue
last_date = df['Order Date'].max()
three_months_ago = last_date - pd.DateOffset(months=3)
recent_orders = df[df['Order Date'] >= three_months_ago]
recent_revenue = recent_orders.groupby('Region')['Sales'].sum().reset_index().rename(columns={'Sales': 'revenue_last_3_months'})

cte_avg_order_value.head(), recent_revenue.head()

(          Customer Name  avg_order_value
 90         Bill Shonely      1166.850333
 156  Christopher Conant      1102.642909
 307      Grant Thornton      1558.535333
 334        Hunter Lopez      1170.299818
 547    Mitch Willingham      1751.292000,
     Region  revenue_last_3_months
 0  Central              46172.031
 1     East              98208.799
 2    South              56064.109
 3     West              80149.888)

In [11]:
# Convert 'Order Date' to month
df['Order_Month'] = df['Order Date'].dt.to_period('M').astype(str)

# Group by Product and Month, count order rows as sales volume
monthly_sales = df.groupby(['Product Name', 'Order_Month']).size().reset_index(name='Sales_Count')

# Sort for trend analysis
monthly_sales = monthly_sales.sort_values(by=['Product Name', 'Order_Month'])


In [12]:
# Pivot to get monthly columns
pivot_sales = monthly_sales.pivot(index='Product Name', columns='Order_Month', values='Sales_Count').fillna(0)

# Get last 3 months from dataset
last_3_months = sorted(pivot_sales.columns)[-3:]

# Filter products with strictly decreasing trend in the last 3 months
def is_decreasing(row):
    vals = row[last_3_months].values
    return vals[0] > vals[1] > vals[2]

decreasing_products = pivot_sales[pivot_sales.apply(is_decreasing, axis=1)]
